<a href="https://colab.research.google.com/github/WasitpolPhanchanokkul/PEA-Summer-Internship/blob/main/V2/E_meter_data_process_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Data

In [ ]:
import pandas as pd
import numpy as np

## Voltage Dataframe

In [ ]:
voltage_df = pd.read_excel('/content/e_meter_data_ฝึกงาน.xlsx',sheet_name='mt_data')

In [ ]:
voltage_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204293 entries, 0 to 204292
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   MT_No         204293 non-null  object 
 1   System Phase  204293 non-null  object 
 2   PoleNo        204293 non-null  object 
 3   Date-Time     204293 non-null  object 
 4   Date          204293 non-null  object 
 5   Month         204293 non-null  int64  
 6   Day           204293 non-null  int64  
 7   Day2          204293 non-null  int64  
 8   Time          204293 non-null  object 
 9   Voltage(V)    204293 non-null  float64
 10  P(kW)         204293 non-null  float64
dtypes: float64(2), int64(3), object(6)
memory usage: 17.1+ MB


In [ ]:
voltage_df.head()

,MT_No,System Phase,PoleNo,Date-Time,Date,Month,Day,Day2,Time,Voltage(V),P(kW)
0,MT0054,C,16M,01-02-2025 00:00:00,01-02-2025,2,1,1,00:00:00,235.08,0.073
1,MT0054,C,16M,01-02-2025 00:15:00,01-02-2025,2,1,1,00:15:00,235.12,0.034
2,MT0054,C,16M,01-02-2025 00:30:00,01-02-2025,2,1,1,00:30:00,235.26,0.017
3,MT0054,C,16M,01-02-2025 00:45:00,01-02-2025,2,1,1,00:45:00,235.75,0.100
4,MT0054,C,16M,01-02-2025 01:00:00,01-02-2025,2,1,1,01:00:00,236.00,0.054


## Distance Dataframe

In [ ]:
distance_df = pd.read_excel('/content/e_meter_data_ฝึกงาน.xlsx',sheet_name='gis_mt_dtm')
distance_df[['MT_No','Distance(M)']]

,MT_No,Distance(M)
0,MT0001,189.8
1,MT0002,189.8
2,MT0003,144.6
3,MT0004,144.6
4,MT0005,85.6
...,...,...
90,MT0094,157.3
91,MT0095,157.3
92,dtmsid1109A,0.0
93,dtmsid1109B,0.0


# Data setup

## Rename

In [ ]:
voltage_df = voltage_df.rename(columns={'MT_No':'mt_no',
                        'System Phase':'system_phase',
                        'PoleNo':'pole_no',
                        'Date':'date',
                        'Date-Time':'date_time',
                        'Voltage(V)':'voltage',
                        'Time':'time'
                        })

In [ ]:
voltage_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204293 entries, 0 to 204292
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   mt_no         204293 non-null  object 
 1   system_phase  204293 non-null  object 
 2   pole_no       204293 non-null  object 
 3   date_time     204293 non-null  object 
 4   date          204293 non-null  object 
 5   Month         204293 non-null  int64  
 6   Day           204293 non-null  int64  
 7   Day2          204293 non-null  int64  
 8   time          204293 non-null  object 
 9   voltage       204293 non-null  float64
 10  P(kW)         204293 non-null  float64
dtypes: float64(2), int64(3), object(6)
memory usage: 17.1+ MB


## change Format date_time

In [ ]:
voltage_df.drop(['date_time','Day'],axis=1,inplace=True)

In [ ]:
voltage_df['date_time'] = pd.to_datetime(
    voltage_df['Day2'].astype(str).str.zfill(2) + '-' +
    voltage_df['Month'].astype(str).str.zfill(2) + '-2025 ' +
    voltage_df['time'],
    format='%d-%m-%Y %H:%M:%S'
)

In [ ]:
voltage_df.drop(['date','Month','Day2'],axis=1,inplace=True)

In [ ]:
voltage_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204293 entries, 0 to 204292
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   mt_no         204293 non-null  object        
 1   system_phase  204293 non-null  object        
 2   pole_no       204293 non-null  object        
 3   time          204293 non-null  object        
 4   voltage       204293 non-null  float64       
 5   P(kW)         204293 non-null  float64       
 6   date_time     204293 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 10.9+ MB


## join distance to voltage

In [ ]:
distance_df.columns = distance_df.columns.str.strip()

# 2. สร้าง dictionary สำหรับ map ค่า distance ตาม mt_no
distance_map = distance_df.set_index('MT_No')['Distance(M)'].to_dict()

# 3. ถ้า df_sam เป็น slice จาก DataFrame อื่น ให้ copy มาก่อน
voltage_df = voltage_df.copy()

# 4. เพิ่มคอลัมน์ใหม่ 'distance' โดย map ตาม mt_no
voltage_df['distance'] = voltage_df['mt_no'].map(distance_map)

# ปัดค่าให้เหลือทศนิยม 1 ตำแหน่ง
voltage_df['distance'] = voltage_df['distance'].round(1)



In [ ]:
voltage_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204293 entries, 0 to 204292
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   mt_no         204293 non-null  object        
 1   system_phase  204293 non-null  object        
 2   pole_no       204293 non-null  object        
 3   time          204293 non-null  object        
 4   voltage       204293 non-null  float64       
 5   P(kW)         204293 non-null  float64       
 6   date_time     204293 non-null  datetime64[ns]
 7   distance      204293 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 12.5+ MB


In [ ]:
voltage_df

,mt_no,system_phase,pole_no,time,voltage,P(kW),date_time,distance
0,MT0054,C,16M,00:00:00,235.080000,0.073000,2025-02-01 00:00:00,162.6
1,MT0054,C,16M,00:15:00,235.120000,0.034000,2025-02-01 00:15:00,162.6
2,MT0054,C,16M,00:30:00,235.260000,0.017000,2025-02-01 00:30:00,162.6
3,MT0054,C,16M,00:45:00,235.750000,0.100000,2025-02-01 00:45:00,162.6
4,MT0054,C,16M,01:00:00,236.000000,0.054000,2025-02-01 01:00:00,162.6
...,...,...,...,...,...,...,...,...
204288,dtmsid1109C,C,Terminal,22:45:00,233.766667,20.866667,2025-02-28 22:45:00,0.0
204289,dtmsid1109C,C,Terminal,23:00:00,234.036667,20.276667,2025-02-28 23:00:00,0.0
204290,dtmsid1109C,C,Terminal,23:15:00,233.516667,20.090000,2025-02-28 23:15:00,0.0
204291,dtmsid1109C,C,Terminal,23:30:00,233.672500,19.465000,2025-02-28 23:30:00,0.0


## Adding Holiday feature

In [ ]:
!pip install holidays

In [ ]:
import holidays

# โหลดวันหยุดราชการไทย ปี 2025
thai_holidays = holidays.Thailand(years=2025)

# แปลง date_time ให้เป็น datetime
# แก้ไขจาก 'date' เป็น 'date_time' เนื่องจากคอลัมน์ 'date' ถูกลบไปแล้ว
voltage_df['date_time'] = pd.to_datetime(voltage_df['date_time'])

# เพิ่มคอลัมน์ใหม่เพื่อตรวจสอบวันหยุด
# แก้ไขจาก 'date' เป็น 'date_time' ในทุกบรรทัดที่เกี่ยวข้อง
voltage_df['is_thai_holiday'] = voltage_df['date_time'].dt.date.isin(thai_holidays)
voltage_df['is_weekend'] = voltage_df['date_time'].dt.weekday >= 5  # เสาร์=5, อาทิตย์=6
voltage_df['is_dayoff'] = voltage_df['is_thai_holiday'] | voltage_df['is_weekend']

In [ ]:
voltage_df = voltage_df.drop(columns=['is_thai_holiday', 'is_weekend'])

In [ ]:
voltage_df

,mt_no,system_phase,pole_no,time,voltage,P(kW),date_time,distance,is_dayoff
0,MT0054,C,16M,00:00:00,235.080000,0.073000,2025-02-01 00:00:00,162.6,True
1,MT0054,C,16M,00:15:00,235.120000,0.034000,2025-02-01 00:15:00,162.6,True
2,MT0054,C,16M,00:30:00,235.260000,0.017000,2025-02-01 00:30:00,162.6,True
3,MT0054,C,16M,00:45:00,235.750000,0.100000,2025-02-01 00:45:00,162.6,True
4,MT0054,C,16M,01:00:00,236.000000,0.054000,2025-02-01 01:00:00,162.6,True
...,...,...,...,...,...,...,...,...,...
204288,dtmsid1109C,C,Terminal,22:45:00,233.766667,20.866667,2025-02-28 22:45:00,0.0,False
204289,dtmsid1109C,C,Terminal,23:00:00,234.036667,20.276667,2025-02-28 23:00:00,0.0,False
204290,dtmsid1109C,C,Terminal,23:15:00,233.516667,20.090000,2025-02-28 23:15:00,0.0,False
204291,dtmsid1109C,C,Terminal,23:30:00,233.672500,19.465000,2025-02-28 23:30:00,0.0,False


In [ ]:
voltage_df[voltage_df.is_dayoff == 1]

,mt_no,system_phase,pole_no,time,voltage,P(kW),date_time,distance,is_dayoff
0,MT0054,C,16M,00:00:00,235.080000,0.073000,2025-02-01 00:00:00,162.6,True
1,MT0054,C,16M,00:15:00,235.120000,0.034000,2025-02-01 00:15:00,162.6,True
2,MT0054,C,16M,00:30:00,235.260000,0.017000,2025-02-01 00:30:00,162.6,True
3,MT0054,C,16M,00:45:00,235.750000,0.100000,2025-02-01 00:45:00,162.6,True
4,MT0054,C,16M,01:00:00,236.000000,0.054000,2025-02-01 01:00:00,162.6,True
...,...,...,...,...,...,...,...,...,...
203845,dtmsid1109C,C,Terminal,22:45:00,234.173333,19.920000,2025-02-23 22:45:00,0.0,True
203846,dtmsid1109C,C,Terminal,23:00:00,232.026667,21.023333,2025-02-23 23:00:00,0.0,True
203847,dtmsid1109C,C,Terminal,23:15:00,232.243333,21.046667,2025-02-23 23:15:00,0.0,True
203848,dtmsid1109C,C,Terminal,23:30:00,232.072500,19.560000,2025-02-23 23:30:00,0.0,True


## Drop MT0018 not use

In [ ]:
# ลบ record ที่ mt_no เป็น MT0018
voltage_df = voltage_df[voltage_df['mt_no'] != 'MT0018']
voltage_df = voltage_df[voltage_df['mt_no'] != 'MT0018'].reset_index(drop=True)


## Drop dtmsid1109A, dtmsid1109B, dtmsid1109C

In [ ]:
voltage_df = voltage_df[voltage_df['mt_no'] != 'dtmsid1109A'].reset_index(drop=True)
voltage_df = voltage_df[voltage_df['mt_no'] != 'dtmsid1109B'].reset_index(drop=True)
voltage_df = voltage_df[voltage_df['mt_no'] != 'dtmsid1109C'].reset_index(drop=True)

## Checking number of record 1 meter per day should be 96

In [ ]:
check_record_df = voltage_df.copy()

In [ ]:
check_record_df['date'] = check_record_df['date_time'].dt.date

In [ ]:
check_record_df

,mt_no,system_phase,pole_no,time,voltage,P(kW),date_time,distance,is_dayoff,date
0,MT0054,C,16M,00:00:00,235.08,0.073000,2025-02-01 00:00:00,162.6,True,2025-02-01
1,MT0054,C,16M,00:15:00,235.12,0.034000,2025-02-01 00:15:00,162.6,True,2025-02-01
2,MT0054,C,16M,00:30:00,235.26,0.017000,2025-02-01 00:30:00,162.6,True,2025-02-01
3,MT0054,C,16M,00:45:00,235.75,0.100000,2025-02-01 00:45:00,162.6,True,2025-02-01
4,MT0054,C,16M,01:00:00,236.00,0.054000,2025-02-01 01:00:00,162.6,True,2025-02-01
...,...,...,...,...,...,...,...,...,...,...
196814,MT0095,C,35M,11:15:00,234.19,0.443667,2025-02-28 11:15:00,157.3,False,2025-02-28
196815,MT0095,C,35M,11:30:00,232.36,0.366000,2025-02-28 11:30:00,157.3,False,2025-02-28
196816,MT0095,C,35M,11:45:00,234.91,0.293667,2025-02-28 11:45:00,157.3,False,2025-02-28
196817,MT0095,C,35M,12:00:00,235.56,0.313667,2025-02-28 12:00:00,157.3,False,2025-02-28


In [ ]:
count_check = check_record_df.groupby(['mt_no', 'date']).size().reset_index(name='record_count')


In [ ]:
count_check = count_check[count_check.record_count != 96]
count_check = count_check.sort_values(by=['date','record_count'])
count_check

,mt_no,date,record_count
531,MT0026,2025-02-28,42
363,MT0020,2025-02-28,43
391,MT0021,2025-02-28,43
475,MT0024,2025-02-28,43
559,MT0027,2025-02-28,43
419,MT0022,2025-02-28,44
447,MT0023,2025-02-28,44
503,MT0025,2025-02-28,44
1259,MT0053,2025-02-28,44
335,MT0019,2025-02-28,45


In [ ]:
check_record_df['date'] = pd.to_datetime(check_record_df['date'])

check_record_df[
    (check_record_df['mt_no'] == 'MT0026') &
    (check_record_df['date'].dt.date == pd.to_datetime('2025-02-28').date())
]


,mt_no,system_phase,pole_no,time,voltage,P(kW),date_time,distance,is_dayoff,date
116912,MT0026,C,31M,00:00:00,235.73,0.0,2025-02-28 00:00:00,20.0,False,2025-02-28
116913,MT0026,C,31M,00:15:00,236.21,0.0,2025-02-28 00:15:00,20.0,False,2025-02-28
116914,MT0026,C,31M,00:30:00,236.46,0.0,2025-02-28 00:30:00,20.0,False,2025-02-28
116915,MT0026,C,31M,00:45:00,234.73,0.0,2025-02-28 00:45:00,20.0,False,2025-02-28
116916,MT0026,C,31M,01:00:00,234.04,0.0,2025-02-28 01:00:00,20.0,False,2025-02-28
116917,MT0026,C,31M,01:15:00,234.12,0.0,2025-02-28 01:15:00,20.0,False,2025-02-28
116918,MT0026,C,31M,01:30:00,233.89,0.0,2025-02-28 01:30:00,20.0,False,2025-02-28
116919,MT0026,C,31M,01:45:00,234.80,0.0,2025-02-28 01:45:00,20.0,False,2025-02-28
116920,MT0026,C,31M,02:00:00,235.16,0.0,2025-02-28 02:00:00,20.0,False,2025-02-28
116921,MT0026,C,31M,02:15:00,235.60,0.0,2025-02-28 02:15:00,20.0,False,2025-02-28


## Drop transaction in 2025-02-28

In [ ]:
import datetime # Add this import


In [ ]:
voltage_df['date'] = voltage_df.date_time.dt.date
# กรองออก โดยแปลงให้ตรงชนิด
voltage_df = voltage_df[voltage_df['date'] != datetime.date(2025, 2, 28)].reset_index(drop=True)
voltage_df

,mt_no,system_phase,pole_no,time,voltage,P(kW),date_time,distance,is_dayoff,date
0,MT0054,C,16M,00:00:00,235.08,0.073000,2025-02-01 00:00:00,162.6,True,2025-02-01
1,MT0054,C,16M,00:15:00,235.12,0.034000,2025-02-01 00:15:00,162.6,True,2025-02-01
2,MT0054,C,16M,00:30:00,235.26,0.017000,2025-02-01 00:30:00,162.6,True,2025-02-01
3,MT0054,C,16M,00:45:00,235.75,0.100000,2025-02-01 00:45:00,162.6,True,2025-02-01
4,MT0054,C,16M,01:00:00,236.00,0.054000,2025-02-01 01:00:00,162.6,True,2025-02-01
...,...,...,...,...,...,...,...,...,...,...
191803,MT0095,C,35M,22:45:00,235.15,0.494667,2025-02-27 22:45:00,157.3,False,2025-02-27
191804,MT0095,C,35M,23:00:00,235.94,0.427333,2025-02-27 23:00:00,157.3,False,2025-02-27
191805,MT0095,C,35M,23:15:00,235.68,0.639667,2025-02-27 23:15:00,157.3,False,2025-02-27
191806,MT0095,C,35M,23:30:00,235.98,0.805667,2025-02-27 23:30:00,157.3,False,2025-02-27


## Change Format Pole delete whitespace

In [ ]:
voltage_df['pole_no'] = voltage_df['pole_no'].str.strip()
voltage_df

,mt_no,system_phase,pole_no,time,voltage,P(kW),date_time,distance,is_dayoff,date
0,MT0054,C,16M,00:00:00,235.08,0.073000,2025-02-01 00:00:00,162.6,True,2025-02-01
1,MT0054,C,16M,00:15:00,235.12,0.034000,2025-02-01 00:15:00,162.6,True,2025-02-01
2,MT0054,C,16M,00:30:00,235.26,0.017000,2025-02-01 00:30:00,162.6,True,2025-02-01
3,MT0054,C,16M,00:45:00,235.75,0.100000,2025-02-01 00:45:00,162.6,True,2025-02-01
4,MT0054,C,16M,01:00:00,236.00,0.054000,2025-02-01 01:00:00,162.6,True,2025-02-01
...,...,...,...,...,...,...,...,...,...,...
191803,MT0095,C,35M,22:45:00,235.15,0.494667,2025-02-27 22:45:00,157.3,False,2025-02-27
191804,MT0095,C,35M,23:00:00,235.94,0.427333,2025-02-27 23:00:00,157.3,False,2025-02-27
191805,MT0095,C,35M,23:15:00,235.68,0.639667,2025-02-27 23:15:00,157.3,False,2025-02-27
191806,MT0095,C,35M,23:30:00,235.98,0.805667,2025-02-27 23:30:00,157.3,False,2025-02-27


## export clean voltage

In [ ]:
from google.colab import files
voltage_df.to_csv('voltage_df.csv', index=False)
files.download('voltage_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Experiment

## EX : 5 Time Series Clustering


## import Data Clean !

Downloading...
From: https://drive.google.com/uc?id=1GpSa0eikaVfeyfABeZ3Xw5tVBYz3Pzc7
To: /content/voltage_df.csv
100% 15.1M/15.1M [00:00<00:00, 28.8MB/s]


In [ ]:
voltage_df = pd.read_csv('/content/voltage_df.csv')
voltage_df

,mt_no,system_phase,pole_no,time,voltage,P(kW),date_time,distance,is_dayoff,date
0,MT0054,C,16M,00:00:00,235.08,0.073000,2025-02-01 00:00:00,162.6,True,2025-02-01
1,MT0054,C,16M,00:15:00,235.12,0.034000,2025-02-01 00:15:00,162.6,True,2025-02-01
2,MT0054,C,16M,00:30:00,235.26,0.017000,2025-02-01 00:30:00,162.6,True,2025-02-01
3,MT0054,C,16M,00:45:00,235.75,0.100000,2025-02-01 00:45:00,162.6,True,2025-02-01
4,MT0054,C,16M,01:00:00,236.00,0.054000,2025-02-01 01:00:00,162.6,True,2025-02-01
...,...,...,...,...,...,...,...,...,...,...
191803,MT0095,C,35M,22:45:00,235.15,0.494667,2025-02-27 22:45:00,157.3,False,2025-02-27
191804,MT0095,C,35M,23:00:00,235.94,0.427333,2025-02-27 23:00:00,157.3,False,2025-02-27
191805,MT0095,C,35M,23:15:00,235.68,0.639667,2025-02-27 23:15:00,157.3,False,2025-02-27
191806,MT0095,C,35M,23:30:00,235.98,0.805667,2025-02-27 23:30:00,157.3,False,2025-02-27


In [ ]:
!pip install tslearn

import pandas as pd
import numpy as np
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.metrics import dtw
import matplotlib.pyplot as plt


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 9.0 MB/s eta 0:00:00


## copy data before to compair with after

In [ ]:
before_df = voltage_df.copy()

## Do all month

#### function

In [ ]:
# ฟังก์ชัน clustering
def cluster_phase_voltage(df, phase_label='A', n_clusters=3):
    df_phase = df[df['system_phase'] == phase_label]
    pivot_df = df_phase.pivot_table(index='mt_no', columns='full_time', values='voltage')
    pivot_df = pivot_df.dropna()

    scaler = TimeSeriesScalerMeanVariance()
    X_scaled = scaler.fit_transform(pivot_df.values[:, :, None])

    model = TimeSeriesKMeans(n_clusters=n_clusters, metric="dtw", random_state=0)
    labels = model.fit_predict(X_scaled)

    pivot_df['cluster'] = labels
    result_df = pivot_df[['cluster']].reset_index()
    return result_df



In [ ]:
# ฟังก์ชันตรวจ mismatch → return เป็น dict: {mt_no: suggested_phase}
def detect_phase_mismatch(df_all, results_dict):
    scaler = TimeSeriesScalerMeanVariance()
    suggestions = {}

    # สร้าง centroid
    centroids = {}
    for phase in ['A', 'B', 'C']:
        df_phase = df_all[df_all['system_phase'] == phase]
        pivot = df_phase.pivot_table(index='mt_no', columns='full_time', values='voltage').dropna()
        if len(pivot) == 0:
            continue
        data = scaler.fit_transform(pivot.values[:, :, None])
        centroid = np.mean(data, axis=0)
        centroids[phase] = centroid

    # ตรวจแต่ละ phase
    for phase in ['A', 'B', 'C']:
        df_phase = df_all[df_all['system_phase'] == phase]
        pivot = df_phase.pivot_table(index='mt_no', columns='full_time', values='voltage').dropna()
        if len(pivot) == 0:
            continue
        data = scaler.fit_transform(pivot.values[:, :, None])

        for i, mt_no in enumerate(pivot.index):
            series = data[i]
            distances = {ph: dtw(series, centroids[ph]) for ph in ['A', 'B', 'C']}
            nearest_phase = min(distances, key=distances.get)

            # เพิ่มลง dict (ถ้าผิด หรือบันทึกว่าเหมือนเดิม)
            suggestions[mt_no] = nearest_phase

    return suggestions


### code

In [ ]:
import pandas as pd
import numpy as np
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.metrics import dtw

# --- เตรียมข้อมูลหลัก ---
voltage_df = voltage_df.rename(columns={'date_time': 'full_time'})
voltage_df['full_time'] = pd.to_datetime(voltage_df['full_time'])
voltage_df['date'] = voltage_df['full_time'].dt.date

# หาวันที่ที่มีครบทั้ง 3 phase
valid_dates = []
for d in voltage_df['date'].unique():
    phases = voltage_df[voltage_df['date'] == d]['system_phase'].unique()
    if all(p in phases for p in ['A', 'B', 'C']):
        valid_dates.append(d)

# --- เริ่มวิเคราะห์ทุกวัน ---
records = []

for current_date in sorted(valid_dates):
    df_day = voltage_df[voltage_df['date'] == current_date].copy()

    # clustering
    result_A = cluster_phase_voltage(df_day, phase_label='A')
    result_B = cluster_phase_voltage(df_day, phase_label='B')
    result_C = cluster_phase_voltage(df_day, phase_label='C')
    results_dict = {'A': result_A, 'B': result_B, 'C': result_C}

    # detect suggestion
    suggest_map = detect_phase_mismatch(df_day, results_dict)

    # รวมผลทีละ row
    for mt_no in df_day['mt_no'].unique():
        df_mt = df_day[df_day['mt_no'] == mt_no]
        current_phase = df_mt['system_phase'].iloc[0]
        suggest_phase = suggest_map.get(mt_no, current_phase)  # ถ้าไม่มีใน suggest ให้ถือว่าไม่ผิด

        full_time = df_mt['full_time'].min()  # เอาเวลาที่เริ่มต้นช่วงนั้น
        records.append({
            'full_time': full_time,
            'mt_no': mt_no,
            'current_phase': current_phase,
            'suggest_phase': suggest_phase
        })

# --- สร้าง experiment_df ---
experiment_df = pd.DataFrame(records)


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11

In [ ]:
experiment_df

,full_time,mt_no,current_phase,suggest_phase
0,2025-02-01,MT0054,C,C
1,2025-02-01,MT0062,C,C
2,2025-02-01,MT0008,B,B
3,2025-02-01,MT0039,B,B
4,2025-02-01,MT0043,A,A
...,...,...,...,...
1993,2025-02-27,MT0091,B,C
1994,2025-02-27,MT0092,C,C
1995,2025-02-27,MT0093,A,C
1996,2025-02-27,MT0094,B,C


In [ ]:
daily_counts = (
    experiment_df.groupby(experiment_df['full_time'].dt.date)
    .size()
    .reset_index(name='count')
    .sort_values(by='full_time')
)
display(daily_counts)


,full_time,count
0,2025-02-01,74
1,2025-02-02,74
2,2025-02-03,74
3,2025-02-04,74
4,2025-02-05,74
5,2025-02-06,74
6,2025-02-07,74
7,2025-02-08,74
8,2025-02-09,74
9,2025-02-10,74


### นำเอา experiment_df ไปกรองอีกรอบ

In [ ]:
experiment_df = experiment_df.sort_values(by = ['full_time','current_phase','mt_no'])
experiment_df.reset_index(drop=True, inplace=True)
experiment_df

,full_time,mt_no,current_phase,suggest_phase
0,2025-02-01,MT0001,A,A
1,2025-02-01,MT0002,A,A
2,2025-02-01,MT0003,A,A
3,2025-02-01,MT0004,A,A
4,2025-02-01,MT0013,A,A
...,...,...,...,...
1993,2025-02-27,MT0070,C,C
1994,2025-02-27,MT0076,C,A
1995,2025-02-27,MT0086,C,C
1996,2025-02-27,MT0092,C,C


In [ ]:
# ดึงเอาแค่ตัวที่ current_phase	กับ suggest_phase ไม่ตรง
mismatch_df = experiment_df[experiment_df['current_phase'] != experiment_df['suggest_phase']]
mismatch_df

,full_time,mt_no,current_phase,suggest_phase
8,2025-02-01,MT0041,A,B
17,2025-02-01,MT0080,A,C
26,2025-02-01,MT0027,B,C
33,2025-02-01,MT0038,B,A
46,2025-02-01,MT0005,C,A
...,...,...,...,...
1982,2025-02-27,MT0042,C,A
1984,2025-02-27,MT0049,C,A
1987,2025-02-27,MT0057,C,A
1988,2025-02-27,MT0059,C,A


In [ ]:
# นับจำนวนผิด phase ต่อ mt_no
mismatch_count = mismatch_df['mt_no'].value_counts()
print("มิเตอร์ที่ผิดบ่อย:", mismatch_count)

# ดูเฟสที่เจอปัญหามากที่สุด
print(mismatch_df['current_phase'].value_counts())


มิเตอร์ที่ผิดบ่อย: mt_no
MT0041    27
MT0027    27
MT0038    27
MT0006    27
MT0005    27
MT0007    27
MT0026    27
MT0076    27
MT0042    27
MT0049    27
MT0057    27
MT0059    27
MT0019    26
MT0050    26
MT0080    25
MT0025    10
MT0020     9
MT0021     9
MT0008     5
MT0036     3
MT0095     2
MT0092     2
MT0029     2
MT0035     2
MT0061     1
MT0034     1
MT0016     1
MT0014     1
MT0040     1
MT0064     1
MT0086     1
MT0072     1
MT0047     1
MT0084     1
MT0090     1
MT0093     1
MT0085     1
MT0091     1
MT0094     1
Name: count, dtype: int64
current_phase
C    334
B     70
A     56
Name: count, dtype: int64


In [ ]:
suspicious_counts = mismatch_df['mt_no'].value_counts().reset_index()
suspicious_counts.columns = ['mt_no', 'mismatch_count']


In [ ]:
top_suspects = suspicious_counts.head(20)
top_suspects = suspicious_counts[suspicious_counts.mismatch_count >= 20]
display(top_suspects)

,mt_no,mismatch_count
0,MT0041,27
1,MT0027,27
2,MT0038,27
3,MT0006,27
4,MT0005,27
5,MT0007,27
6,MT0026,27
7,MT0076,27
8,MT0042,27
9,MT0049,27


### Result

In [ ]:
from collections import Counter

# รายชื่อ mt_no ที่สนใจ
target_mt_no = top_suspects['mt_no'].tolist()

# เฉพาะ record ที่ mismatch และอยู่ใน target_mt_no
mismatch_df = experiment_df[
    (experiment_df['current_phase'] != experiment_df['suggest_phase']) &
    (experiment_df['mt_no'].isin(target_mt_no))
]

# รวม phase ที่ผิด → กลุ่มตาม current_phase
report_dict = {}

for phase in ['A', 'B', 'C']:
    df_phase = mismatch_df[mismatch_df['current_phase'] == phase]

    # รวมแต่ละ mt_no และหาว่าควรเป็น suggest_phase ใดบ่อยที่สุด
    mt_phase_map = (
        df_phase
        .groupby('mt_no')['suggest_phase']
        .agg(lambda x: Counter(x).most_common(1)[0][0])  # suggest ที่โหวตมากสุด
        .reset_index()
    )

    report_dict[phase] = mt_phase_map

# พิมพ์แบบฟอร์แมต
for phase in ['A', 'B', 'C']:
    print(f"--- ตรวจ Phase {phase} ---")
    df = report_dict[phase]
    if df.empty:
        print("- ไม่พบความผิดปกติ")
        continue

    for _, row in df.iterrows():
        print(f"- {row['mt_no']} อาจอยู่ผิด phase ({phase}) → ควรเป็น Phase {row['suggest_phase']}")


--- ตรวจ Phase A ---
- MT0041 อาจอยู่ผิด phase (A) → ควรเป็น Phase B
- MT0080 อาจอยู่ผิด phase (A) → ควรเป็น Phase C
--- ตรวจ Phase B ---
- MT0027 อาจอยู่ผิด phase (B) → ควรเป็น Phase C
- MT0038 อาจอยู่ผิด phase (B) → ควรเป็น Phase A
--- ตรวจ Phase C ---
- MT0005 อาจอยู่ผิด phase (C) → ควรเป็น Phase A
- MT0006 อาจอยู่ผิด phase (C) → ควรเป็น Phase A
- MT0007 อาจอยู่ผิด phase (C) → ควรเป็น Phase A
- MT0019 อาจอยู่ผิด phase (C) → ควรเป็น Phase A
- MT0026 อาจอยู่ผิด phase (C) → ควรเป็น Phase B
- MT0042 อาจอยู่ผิด phase (C) → ควรเป็น Phase A
- MT0049 อาจอยู่ผิด phase (C) → ควรเป็น Phase A
- MT0050 อาจอยู่ผิด phase (C) → ควรเป็น Phase A
- MT0057 อาจอยู่ผิด phase (C) → ควรเป็น Phase A
- MT0059 อาจอยู่ผิด phase (C) → ควรเป็น Phase A
- MT0076 อาจอยู่ผิด phase (C) → ควรเป็น Phase A


In [ ]:
# กรองเฉพาะแถวที่ mt_no อยู่ใน top_suspects และ phase_diff ≠ 0
target_mt_no = top_suspects['mt_no'].tolist()
filtered_df = mismatch_df[mismatch_df['mt_no'].isin(target_mt_no)]

# Group by ตาม mt_no, current_phase, suggest_phase แล้วนับจำนวน
report_to_explore = (
    filtered_df
    .groupby(['mt_no', 'current_phase', 'suggest_phase'])
    .size()
    .reset_index(name='count')
    .rename(columns={'current_phase': 'old_phase', 'suggest_phase': 'new_phase'})
)

# แสดงผล
# เก็บเฉพาะรายการที่ count มากที่สุดต่อ mt_no
report_to_explore = (
    report_to_explore
    .sort_values(by='count', ascending=False)
    .drop_duplicates(subset='mt_no', keep='first')
    .sort_values(by=['old_phase', 'count'], ascending=[True, False])
)

display(report_to_explore)




,mt_no,old_phase,new_phase,count
8,MT0041,A,B,27
15,MT0080,A,C,25
7,MT0038,B,A,27
6,MT0027,B,C,18
0,MT0005,C,A,27
1,MT0006,C,A,27
2,MT0007,C,A,27
4,MT0026,C,B,27
10,MT0049,C,A,27
9,MT0042,C,A,27


In [ ]:
report_to_explore = report_to_explore.reset_index(drop=True)
report_to_explore

,mt_no,old_phase,new_phase,count
0,MT0041,A,B,27
1,MT0080,A,C,25
2,MT0038,B,A,27
3,MT0027,B,C,18
4,MT0005,C,A,27
5,MT0006,C,A,27
6,MT0007,C,A,27
7,MT0026,C,B,27
8,MT0049,C,A,27
9,MT0042,C,A,27


In [ ]:
after_df = voltage_df.copy()

In [ ]:
after_df

,mt_no,system_phase,pole_no,time,voltage,P(kW),full_time,distance,is_dayoff,date
0,MT0054,C,16M,00:00:00,235.08,0.073000,2025-02-01 00:00:00,162.6,True,2025-02-01
1,MT0054,C,16M,00:15:00,235.12,0.034000,2025-02-01 00:15:00,162.6,True,2025-02-01
2,MT0054,C,16M,00:30:00,235.26,0.017000,2025-02-01 00:30:00,162.6,True,2025-02-01
3,MT0054,C,16M,00:45:00,235.75,0.100000,2025-02-01 00:45:00,162.6,True,2025-02-01
4,MT0054,C,16M,01:00:00,236.00,0.054000,2025-02-01 01:00:00,162.6,True,2025-02-01
...,...,...,...,...,...,...,...,...,...,...
191803,MT0095,C,35M,22:45:00,235.15,0.494667,2025-02-27 22:45:00,157.3,False,2025-02-27
191804,MT0095,C,35M,23:00:00,235.94,0.427333,2025-02-27 23:00:00,157.3,False,2025-02-27
191805,MT0095,C,35M,23:15:00,235.68,0.639667,2025-02-27 23:15:00,157.3,False,2025-02-27
191806,MT0095,C,35M,23:30:00,235.98,0.805667,2025-02-27 23:30:00,157.3,False,2025-02-27


In [ ]:
# สร้าง mapping จาก mt_no → new_phase
phase_map = report_to_explore.set_index('mt_no')['new_phase'].to_dict()

# แทนค่าใน after_df เฉพาะ mt_no ที่มีใน mapping
after_df['system_phase'] = after_df['mt_no'].map(phase_map).fillna(after_df['system_phase'])
after_df

,mt_no,system_phase,pole_no,time,voltage,P(kW),full_time,distance,is_dayoff,date
0,MT0054,C,16M,00:00:00,235.08,0.073000,2025-02-01 00:00:00,162.6,True,2025-02-01
1,MT0054,C,16M,00:15:00,235.12,0.034000,2025-02-01 00:15:00,162.6,True,2025-02-01
2,MT0054,C,16M,00:30:00,235.26,0.017000,2025-02-01 00:30:00,162.6,True,2025-02-01
3,MT0054,C,16M,00:45:00,235.75,0.100000,2025-02-01 00:45:00,162.6,True,2025-02-01
4,MT0054,C,16M,01:00:00,236.00,0.054000,2025-02-01 01:00:00,162.6,True,2025-02-01
...,...,...,...,...,...,...,...,...,...,...
191803,MT0095,C,35M,22:45:00,235.15,0.494667,2025-02-27 22:45:00,157.3,False,2025-02-27
191804,MT0095,C,35M,23:00:00,235.94,0.427333,2025-02-27 23:00:00,157.3,False,2025-02-27
191805,MT0095,C,35M,23:15:00,235.68,0.639667,2025-02-27 23:15:00,157.3,False,2025-02-27
191806,MT0095,C,35M,23:30:00,235.98,0.805667,2025-02-27 23:30:00,157.3,False,2025-02-27


# Compair Before and after

## change phase success

In [ ]:
before_df[before_df.mt_no == 'MT0041']

,mt_no,system_phase,pole_no,time,voltage,P(kW),date_time,distance,is_dayoff,date
67392,MT0041,A,35M,00:00:00,234.68,0.246,2025-02-01 00:00:00,157.3,True,2025-02-01
67393,MT0041,A,35M,00:15:00,234.28,0.288,2025-02-01 00:15:00,157.3,True,2025-02-01
67394,MT0041,A,35M,00:30:00,235.07,0.210,2025-02-01 00:30:00,157.3,True,2025-02-01
67395,MT0041,A,35M,00:45:00,235.15,0.231,2025-02-01 00:45:00,157.3,True,2025-02-01
67396,MT0041,A,35M,01:00:00,235.02,0.201,2025-02-01 01:00:00,157.3,True,2025-02-01
...,...,...,...,...,...,...,...,...,...,...
69979,MT0041,A,35M,22:45:00,234.80,0.140,2025-02-27 22:45:00,157.3,False,2025-02-27
69980,MT0041,A,35M,23:00:00,235.31,0.186,2025-02-27 23:00:00,157.3,False,2025-02-27
69981,MT0041,A,35M,23:15:00,234.82,0.232,2025-02-27 23:15:00,157.3,False,2025-02-27
69982,MT0041,A,35M,23:30:00,235.25,0.204,2025-02-27 23:30:00,157.3,False,2025-02-27


In [ ]:
after_df[after_df.mt_no == 'MT0041']

,mt_no,system_phase,pole_no,time,voltage,P(kW),full_time,distance,is_dayoff,date
67392,MT0041,B,35M,00:00:00,234.68,0.246,2025-02-01 00:00:00,157.3,True,2025-02-01
67393,MT0041,B,35M,00:15:00,234.28,0.288,2025-02-01 00:15:00,157.3,True,2025-02-01
67394,MT0041,B,35M,00:30:00,235.07,0.210,2025-02-01 00:30:00,157.3,True,2025-02-01
67395,MT0041,B,35M,00:45:00,235.15,0.231,2025-02-01 00:45:00,157.3,True,2025-02-01
67396,MT0041,B,35M,01:00:00,235.02,0.201,2025-02-01 01:00:00,157.3,True,2025-02-01
...,...,...,...,...,...,...,...,...,...,...
69979,MT0041,B,35M,22:45:00,234.80,0.140,2025-02-27 22:45:00,157.3,False,2025-02-27
69980,MT0041,B,35M,23:00:00,235.31,0.186,2025-02-27 23:00:00,157.3,False,2025-02-27
69981,MT0041,B,35M,23:15:00,234.82,0.232,2025-02-27 23:15:00,157.3,False,2025-02-27
69982,MT0041,B,35M,23:30:00,235.25,0.204,2025-02-27 23:30:00,157.3,False,2025-02-27


 # export output

In [ ]:
#report_to_explore to excel

from google.colab import files
report_to_explore.to_excel('report_to_explore.xlsx', index=False)
files.download('report_to_explore.xlsx')


In [ ]:
#report_to_explore to csv
from google.colab import files
report_to_explore.to_csv('report_to_explore.csv', index=False)
files.download('report_to_explore.csv')

In [ ]:

from google.colab import files
before_df.to_csv('before_df.csv', index=False)
files.download('before_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
after_df.to_excel('after_df_change_phase.xlsx', index=False)
files.download('after_df_change_phase.xlsx')

In [ ]:
from google.colab import files
after_df.to_csv('after_df_change_phase.csv', index=False)
files.download('after_df_change_phase.csv')